# Boosting Monocular Depth Estimation Models to High-Resolution via Content-Adaptive Multi-Resolution Merging

This Colab Notebook contains an inference on the implementation from https://github.com/compphoto/BoostingMonocularDepth.

It supports using [MiDas-v2][1] and [LeRes][2] but not [SGRnet][3].


### Instructions
0. Ensure using a GPU by setting "Runtime/change runtime type" to GPU
1. Run code section 1
2. Put your test images inside /content/inputs
3. Run code section 2 :  it will download the needed model weights for both [MiDas-v2][1] and [LeRes][2]. 

5. Run code section 3
4. Run code section "Ours with MiDas" or "Ours with LeRes" to generate the results 
3. Results will be generated in /content/outputs_midas or /content/outputs_leres folders. 


[1]: https://github.com/intel-isl/MiDaS/tree/v2
[2]: https://github.com/aim-uofa/AdelaiDepth/tree/main/LeReS
[3]: https://github.com/KexianHust/Structure-Guided-Ranking-Loss

The code assumes ground truth is disparity, not depth.

In [1]:
# Code section 1

!mkdir -p inputs

# output folders
!mkdir -p outputs_midas
!mkdir -p outputs_leres

In [ ]:
# Code section 2


# Clone git repo
!git clone https://github.com/krisrjohnson/BoostingMonocularDepth.git

# !wget https://sfu.ca/~yagiz/CVPR21/latest_net_G.pth
!gdown https://drive.google.com/u/0/uc?id=1cU2y-kMbt0Sf00Ns4CN2oO9qPJ8BensP&export=download

# Downloading merge model weights
!mkdir -p /content/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/
!mv latest_net_G.pth /content/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/

# Downloading Midas weights
# !gdown https://drive.google.com/uc?id=1nqW_Hwj86kslfsXR7EnXpEWdO2csz1cC
!gdown https://drive.google.com/uc?id=1kEtwYckofeZbpg2I1_xaP60IZ3yasS1p&export=download
!mv model.pt /content/BoostingMonocularDepth/midas/

# # Downloading LeRes weights
!wget https://cloudstor.aarnet.edu.au/plus/s/lTIJF4vrvHCAI31/download
!mv download /content/BoostingMonocularDepth/res101.pth

In [ ]:
# sample image
!gdown https://drive.google.com/uc?id=1jxxo5xKAvMC7U7BOmRk1wsl-u4c0iM68&export=download
!mv Adirondack-perfect_im0.png /content/inputs/Adirondack-perfect_im0.png

# !gdown https://drive.google.com/file/d/1IPqFLjXwSEOVqKzus9Asf2aL-hvL1mQ2&export=download
# !mv Bicycle1-perfect-disp0.pfm /content/gt_depth/Bicycle1-perfect-im0.png

In [ ]:
# Code section 3
%cd BoostingMonocularDepth/

> After execution of Code section 3 you can run the following code sections multiple times to generate results but **do not run the previous code sections**. If you did so by mistake use "Runtime/ Reset factory runtime" and then start from step 0.

In [ ]:
# Running the method using MiDas
!python run.py --colorize_results --Final --data_dir /content/inputs --output_dir  /content/outputs_midas/ --depthNet 0

In [ ]:
# Running the method using LeRes
!python run.py --Final --data_dir /content/inputs --output_dir  /content/outputs_leres/ --depthNet 2

# Display Results

In [5]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from pathlib import Path

In [ ]:
img_files = sorted(list(Path('/content/outputs_midas/').iterdir()))
fig, ax = plt.subplots(nrows=len(img_files), ncols=1)

for i, img_file in enumerate(img_files):
  img = Image.open(img_file)
  img = np.array(img).astype('uint32')
  if len(img_files) > 1:
    ax[i].imshow(img)
    ax[i].set_xlabel(img_file.stem)
  else:
    ax.imshow(img)
    ax.set_xlabel(img_file.stem)